In [1]:
import os
import sys
import json
import time
import re
from shutil import copy
from subprocess import check_output
import pandas
from pandas import DataFrame


from execo import Process, SshProcess, Put, Remote
from execo_engine import Engine, logger, ParamSweeper, sweep
from execo_g5k import oarsub, oardel, OarSubmission, get_current_oar_jobs, get_oar_job_subnets, get_oar_job_nodes, get_oar_job_info, \
Deployment, deploy

2024-01-25 14:06:15,791 WARNING: terminated: <SshProcess('/home/mjay/FL-G5K-Test/xav_read_power.py', Host('estats-5.toulouse.grid5000.fr'), name=/home/mjay/FL-G5K-Test/xav_read_powe..., started=True, start_date=2024-01-25 14:06:14+01:00, ended=True, end_date=2024-01-25 14:06:15+01:00, killed=False, error=False, error_reason=None, timeouted=False, expect_fail=False, write_error=False, exit_code=32256, ok=False, pid=61160, real cmd=('ssh', '-tt', '-o', 'BatchMode=yes', '-o', 'PasswordAuthentication=no', '-o', 'StrictHostKeyChecking=no', '-o', 'UserKnownHostsFile=/dev/null', '-o', 'ConnectTimeout=20', 'estats-5.toulouse.grid5000.fr', '/home/mjay/FL-G5K-Test/xav_read_power.py'))>
stdout:
bash: /home/mjay/FL-G5K-Test/xav_read_power.py: Permission denied

stderr:
Connection to estats-5.toulouse.grid5000.fr closed.

2024-01-25 14:09:23,728 WARNING: terminated: <SshProcess('./jetson_monitoring_energy.py', Host('estats-5.toulouse.grid5000.fr'), name=./jetson_monitoring_energy.py, started=True, 

In [7]:
jobname="fl-measure"
nodecount=2
walltime="02:00:00"
resources_selection="-t exotic -p estats"
site="toulouse"
force_redeploy=False
environment_dsc_file='./images/fl_jetson_image.yaml'

In [3]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname,
                            queue='testing')
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 448428


In [4]:
nodes = get_oar_job_nodes(jobid, site)
nodes.sort(key=lambda n: n.address)
nodes

[Host('estats-5.toulouse.grid5000.fr'), Host('estats-8.toulouse.grid5000.fr')]

In [5]:
server=nodes[0]
clients=nodes[1:]
print("Server:{} \n Clients: {}".format(server,clients))

Server:Host('estats-5.toulouse.grid5000.fr') 
 Clients: [Host('estats-8.toulouse.grid5000.fr')]


In [8]:
deployment = Deployment(hosts=nodes, env_file=os.path.abspath(environment_dsc_file))
deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])
print("Deployement status:\n* ok: {}\n* failed: {}".format(deploy_ok, deploy_failed))

toulouse: Deployment #D-f3fdc486-b2ab-400a-928c-2cdd92788fff startedtoulouse: toulouse: Grab the key file /home/mjay/.ssh/authorized_keystoulouse: toulouse: Grab the tarball file http://public.toulouse.grid5000.fr/~tunguyen/fl_jetson_image.tar.zsttoulouse: toulouse: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgztoulouse: toulouse: Using classical reboot instead of kexec (SetDeploymentMiniOS) because the last deployment is not trustedtoulouse: toulouse: Launching a deployment of fl_jetson_image:mjay:aarch64:2024011717 on estats-[5,8].toulouse.grid5000.frtoulouse: toulouse: Performing a Deploy[SetDeploymentMiniOSUntrusted] steptoulouse: toulouse:   switch_pxetoulouse: toulouse: [dbg] -------------------------toulouse: toulouse: [dbg] NODE: estats-5.toulouse.grid5000.frtoulouse: toulouse: [dbg] -------------------------toulouse: toulouse: [dbg] COMMAND: KADEPLOY_BOOTLOADER_NO_GRUB_FROM_DEST="1" KADEPLOY_CLUSTER="estats" KADEPLOY_VERSION="3.8.2.stable" KADEPL

toulouse: Connection to toulouse.grid5000.fr closed.toulouse: 

Deployement status:
* ok: {'estats-5.toulouse.grid5000.fr', 'estats-8.toulouse.grid5000.fr'}
* failed: set()


# Defining hyper parameters

In [61]:
from datetime import datetime

exp_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

dir = "/home/mjay/FL-G5K-Test/"
jetson_sensor_monitor = f"{dir}/jetson_monitoring_energy.py"
result_folder = f"{dir}/outputs/{exp_datetime}"

# Create hyperparameters folder
hyperparams = {}
hyperparams["result_folder"]=result_folder
hyperparams["exp_datetime"]=exp_datetime

# TODO: Add all hyperparameters from config

# Start monitoring the energy

Below is the code to use Simon's code. For now I don't think it gives us more information than my code so let's keep using mine (so we don't have to change the code for plotting etc). We'll try to use ALUMET once it's ready instead.
The uncomment cell start my monitoring code in background.

In [ ]:
# Start the process
# process = SshProcess("python3 /home/mjay/FL-G5K-Test/xav_read_power.py", host=server, connection_params={'user':'root'})
# process.start()

In [ ]:
# To get the data (later)
# import requests
# request_start = time.strftime('%Y-%m-%dT%H:%M:%S', time.localtime(time.time() - 1000))
# request_stop = time.strftime('%Y-%m-%dT%H:%M:%S', time.localtime(time.time() -5))
# node = "estats-5"

# url = f"https://api.grid5000.fr/stable/sites/toulouse/metrics?nodes={node}&start_time={request_start}&end_time={request_stop}"
# # url = "https://api.grid5000.fr/stable/sites/toulouse/metrics/?"
# r = requests.get(url, verify=False).json() 

In [53]:
command = f"python3 {jetson_sensor_monitor} --log-dir {result_folder} --log-csv energy.csv"
jtop_process = SshProcess(command, host=server, connection_params={'user':'root'})
jtop_process.start()

SshProcess('python3 /home/mjay/FL-G5K-Test/jetson_monitoring_energy.py --log-dir /home/mjay/FL-G5K-Test//outputs/ --log-csv energy.csv', Host('estats-5.toulouse.grid5000.fr'), connection_params={'user': 'root'})

In [57]:
jtop_process.stdout

'DEBUG - 2024-01-25 14:57:31,072 - jtop.py - 133 : Loaded jetson_variables variables\r\nINFO - 2024-01-25 14:57:31,357 - jetson_monitoring_energy.py - 9 : Starting monitoring energy using jtop.\r\nINFO - 2024-01-25 14:57:31,361 - jetson_monitoring_energy.py - 13 : Saving the power data in /home/mjay/FL-G5K-Test//outputs/energy.csv\r\n'

# RUN EXPERIMENT

In [58]:
def get_host_ip(hostname):
    command = f"hostname -I"
    process = SshProcess(command, host=hostname)
    process.run()
    if process.ok:
        ip_address = process.stdout.strip()
        process.kill()
        return ip_address
    else:
        process.kill()
        return f"Failed to get IP for {hostname}"

# Example usage
hostname = "your_host"
ip_address = get_host_ip(server)
print(f"IP address for {hostname} is {ip_address}")

IP address for your_host is 172.16.121.5 172.17.0.1 2001:660:4406:900:a::5


In [62]:
run_server=SshProcess(f"cd {dir}; bash run_server.sh {exp_datetime}", server, connection_params={'user':'root'})
clients_procs = []
for (host,cid) in zip(clients,range(len(clients))):
    run_client=SshProcess(f"cd {dir}; bash run_client.sh {cid} {exp_datetime}", host, connection_params={'user':'root'})
    clients_procs.append(run_client)

In [63]:
run_server.start()
#print(f"Output from {server}:\n{run_server.stdout}")

SshProcess('cd /home/mjay/FL-G5K-Test/; bash run_server.sh 2024-01-25_15-25-39', Host('estats-5.toulouse.grid5000.fr'), connection_params={'user': 'root'})

In [64]:
for run_client in clients_procs:
    run_client.start()
    #print(f"Output from {run_client.host}:\n{run_client.stdout}")

In [65]:
print(f"Output from server {server}:\n{run_server.stdout}")
print(f"Output from client {clients_procs[0]}:\n{clients_procs[0].stdout}")

Output from server Host('estats-5.toulouse.grid5000.fr'):
Jetson sensor running with pid 17184
DEBUG - 2024-01-25 15:26:12,834 - jtop.py - 133 : Loaded jetson_variables variables
INFO - 2024-01-25 15:26:13,284 - jetson_monitoring_energy.py - 9 : Starting monitoring energy using jtop.
INFO - 2024-01-25 15:26:13,285 - jetson_monitoring_energy.py - 13 : Saving the power data in /tmp/results_energy/2024-01-25/15-26-12/energy.csv
start_server DATE 2024/01/25T15:26:15.219292
/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.8/dist-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(

Output from client <SshProce

In [67]:
run_server.wait()

SshProcess('cd /home/mjay/FL-G5K-Test/; bash run_server.sh 2024-01-25_15-25-39', Host('estats-5.toulouse.grid5000.fr'), connection_params={'user': 'root'})

In [160]:
run_server.kill()
[run_client.kill() for run_client in clients_procs]

[SshProcess('cd /home/tunguyen/jetson-test/; bash run_client.sh 0', Host('estats-10.toulouse.grid5000.fr'), connection_params={'user': 'root'}),
 SshProcess('cd /home/tunguyen/jetson-test/; bash run_client.sh 1', Host('estats-3.toulouse.grid5000.fr'), connection_params={'user': 'root'}),
 SshProcess('cd /home/tunguyen/jetson-test/; bash run_client.sh 2', Host('estats-5.toulouse.grid5000.fr'), connection_params={'user': 'root'}),
 SshProcess('cd /home/tunguyen/jetson-test/; bash run_client.sh 3', Host('estats-6.toulouse.grid5000.fr'), connection_params={'user': 'root'}),
 SshProcess('cd /home/tunguyen/jetson-test/; bash run_client.sh 4', Host('estats-7.toulouse.grid5000.fr'), connection_params={'user': 'root'}),
 SshProcess('cd /home/tunguyen/jetson-test/; bash run_client.sh 5', Host('estats-9.toulouse.grid5000.fr'), connection_params={'user': 'root'})]

In [68]:
oardel([(jobid,site)])